**Test / Example note book of the functions executed in card_analyzer.py**

In [1]:
%load_ext autoreload
%autoreload 2

### Load example dataset

In [2]:
import os
import sys
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from src import *
from card import *

Card classes and methods successfully imported


In [4]:
# Set path of the folder containing dataset
dataset_FolderPath = Path.cwd().parent / 'data' # @dev TBC before each use

# Set path of the File
dataset_FileName = 'AllPrintings.json'
dataset_FilePath = dataset_FolderPath / dataset_FileName

In [5]:
# Load all datasets
data = pd.read_json(dataset_FilePath)
data = data.iloc[2:]['data'] # 2 first rows of JSON files are metadata

In [6]:
# Load a test dataset
set_code = 'OTJ'
cards = load_set(data, set_code, restriction='base_set')
cards

,name,keywords,manaValue,manaCost,colorIdentity,power,toughness,rarity,types,text
0,Another Round,NaN,3.0,{X}{X}{2}{W},[W],0.0,0.0,rare,[Sorcery],"Exile any number of creatures you control, the..."
1,Archangel of Tithes,[Flying],4.0,{1}{W}{W}{W},[W],3.0,5.0,mythic,[Creature],Flying\nAs long as Archangel of Tithes is unta...
2,Armored Armadillo,[Ward],1.0,{W},[W],0.0,4.0,common,[Creature],Ward {1} (Whenever this creature becomes the t...
3,Aven Interrupter,"[Flash, Flying, Plot]",3.0,{1}{W}{W},[W],2.0,2.0,rare,[Creature],"Flash\nFlying\nWhen Aven Interrupter enters, e..."
4,Bounding Felidar,[Saddle],6.0,{5}{W},[W],4.0,7.0,uncommon,[Creature],Whenever Bounding Felidar attacks while saddle...
...,...,...,...,...,...,...,...,...,...,...
266,Botanical Sanctum,NaN,0.0,NaN,"[G, U]",0.0,0.0,rare,[Land],Botanical Sanctum enters tapped unless you con...
267,Concealed Courtyard,NaN,0.0,NaN,"[B, W]",0.0,0.0,rare,[Land],Concealed Courtyard enters tapped unless you c...
268,Inspiring Vantage,NaN,0.0,NaN,"[R, W]",0.0,0.0,rare,[Land],Inspiring Vantage enters tapped unless you con...
269,Spirebluff Canal,NaN,0.0,NaN,"[R, U]",0.0,0.0,rare,[Land],Spirebluff Canal enters tapped unless you cont...


In [7]:
# Test for 1 card

test_card0 = Card(cards.iloc[3])
"""
Aven Interrupter
Flash
Flying
When Aven Interrupter enters, exile target spell. It becomes plotted. (Its owner may cast it as a sorcery on a later turn without paying its mana cost.)
Spells your opponents cast from graveyards or from exile cost {2} more to cast.
"""
test_card0.show()
print(test_card0.show()['text'])
print(test_card0.is_type(['Creature']))
print(test_card0.is_body())
print(test_card0.effects.is_targeting())


Flash
Flying
When Aven Interrupter enters, exile target spell. It becomes plotted. (Its owner may cast it as a sorcery on a later turn without paying its mana cost.)
Spells your opponents cast from graveyards or from exile cost {2} more to cast.
True
True
True


In [8]:
# Test for for filtering several cards

# cards[cards.apply(lambda x: Card(x).is_body(), axis=1)]
# cards[cards.apply(lambda x: Card(x).is_interaction(), axis=1)]
cards[cards.apply(lambda x: Card(x).is_mana_producer(), axis=1)]

,name,keywords,manaValue,manaCost,colorIdentity,power,toughness,rarity,types,text
86,Fake Your Own Death,[Treasure],2.0,{1}{B},[B],0.0,0.0,common,[Instant],"Until end of turn, target creature gets +2/+0 ..."
109,Treasure Dredger,[Treasure],2.0,{1}{B},[B],2.0,2.0,uncommon,[Creature],"{1}, {T}, Pay 1 life: Create a Treasure token...."
134,Mine Raider,"[Trample, Treasure]",3.0,{2}{R},[R],3.0,2.0,common,[Creature],"Trample\nWhen Mine Raider enters, if you contr..."
139,Reckless Lackey,"[First strike, Haste, Treasure]",1.0,{R},[R],1.0,2.0,common,[Creature],"First strike, haste\n{2}{R}, Sacrifice Reckles..."
142,Rodeo Pyromancers,NaN,4.0,{3}{R},[R],3.0,4.0,common,[Creature],"Whenever you cast your first spell each turn, ..."
167,Hardbristle Bandit,NaN,2.0,{1}{G},[G],1.0,1.0,common,[Creature],{T}: Add one mana of any color.\nWhenever you ...
168,Intrepid Stablemaster,[Reach],2.0,{1}{G},[G],2.0,2.0,uncommon,[Creature],Reach\n{T}: Add {G}.\n{T}: Add two mana of any...
172,Outcaster Trailblazer,[Plot],3.0,{2}{G},[G],4.0,2.0,rare,[Creature],"When Outcaster Trailblazer enters, add one man..."
227,"Roxanne, Starfall Savant",NaN,5.0,{3}{R}{G},"[G, R]",4.0,3.0,rare,[Creature],"Whenever Roxanne, Starfall Savant enters or at..."
230,"Selvala, Eager Trailblazer",[Vigilance],4.0,{2}{G}{W},"[G, W]",4.0,5.0,mythic,[Creature],"Vigilance\nWhenever you cast a creature spell,..."


In [ ]:
# test manaprod features
test_card1 = Card(cards.iloc[109])
test_card1.show()

"""
Treasure Dredger
'{1}, {T}, Pay 1 life: Create a Treasure token. (It\'s an artifact with "{T}, Sacrifice this artifact: Add one mana of any color.")'
"""
print(test_card1.is_mana_producer())
print(test_card1.is_type(['Creature']))
print(test_card1.manaprod.is_type(['Creature']))

test_card1.manaprod.producer_type()
test_card1.manaprod.mana_produced()
print(test_card1.manaprod.manaprod_features['manaprod_type'])
print(test_card1.manaprod.manaprod_features['mana_produced'])


True
True
True
Treasures
{'W': 0, 'U': 0, 'B': 0, 'R': 0, 'G': 0, 'C': 0, 'ALL': 1}


In [ ]:
# test body features
test_card2 = Card(cards.iloc[37])
test_card2.show()

"""
Wanted Griffin
Flying
When Wanted Griffin dies, create a 1/1 red Mercenary creature token with "{T}: Target creature you control gets +1/+0 until end of turn. Activate only as a sorcery.
"""
print(test_card2.is_body())
print(test_card2.body.is_evasive())




True
True
['Flying']


In [185]:
# getBodyStats()

def isQuasiBody(card):
    return False

def getBodyStats(card):

    # pass the function it is not a body
    if not (isBody(card) | isQuasiBody(card)):
        return print("Not a body")

    # Filters
    ## 1. creature only
    filter1 = (
        isType(card['types'], ['Creature']) 
        & (not createsToken(card['text'])) 
    )
    ## 2. creature that creates a creature token on ETB
    filter2 = (
        isType(card['types'], ['Creature']) 
        & createsToken(card['text']) 
        & isETB(card['text'])
    ) 
    ## 3. permanent that creates a creature token on ETB
    filter3 = (
        isPermanent(card['types']) 
        & createsToken(card['text']) 
        & isETB(card['text'])
    ) 
    ## 4. instant / sorcery spell that creates a creature token
    filter4 = (
        isType(card['types'], ['Instant', 'Sorcery']) 
        & createsToken(card['text']) 
    ) 

    def findTokenPowerToughness(text):
        pattern = r'creat(e|es).*?(\b(?:\d+|X)/(?:\d+|X)\b).*?creature token'
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        
        if match:
            # Extract the power/toughness string (e.g., "2/2" or "X/3")
            power, toughness = match.group(2).split('/')
            
            # Convert to integer if possible, or return 0 if 'X' is found
            power_int = int(power) if power.isdigit() else 0
            toughness_int = int(toughness) if toughness.isdigit() else 0
            
            return power_int, toughness_int
        else:
            return 0, 0  # Return (0, 0) if no match is found
    
    if filter1==True:
        bdType = 'Creature'
        bdManaValue = card['manaValue']
        bdPower = card['power']
        bdToughness = card['toughness']

    if filter2==True:
        bdType = 'Creature with ETB creature token'
        bdManaValue = card['manaValue']

        p, t = findTokenPowerToughness(card['text'])
        bdPower = card['power'] + p
        bdToughness = card['toughness'] + t

    if filter3==True:
        bdType = 'Non-creature with ETB creature token'
        bdManaValue = card['manaValue']
        
        p, t = findTokenPowerToughness(card['text'])
        bdPower = card['power'] + p
        bdToughness = card['toughness'] + t

    if filter4==True:
        bdType = 'Non-permanent with creature token'
        bdManaValue = card['manaValue']
        
        p, t = findTokenPowerToughness(card['text'])
        bdPower = p
        bdToughness = t

    return bdType, bdManaValue, bdPower, bdToughness

# Test
c = cards.loc[17]
print(c.text)

getBodyStats(c)

When Lassoed by the Law enters, exile target nonland permanent an opponent controls until Lassoed by the Law leaves the battlefield.
When Lassoed by the Law enters, create a 1/1 red Mercenary creature token with "{T}: Target creature you control gets +1/+0 until end of turn. Activate only as a sorcery."
yay
success


('Non-creature with ETB creature token', 4.0, nan, nan)